# Argumentation Abstraite (Dung)

**Navigation**: [← Tweety-4-Belief-Revision](Tweety-4-Belief-Revision.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-6-Structured-Argumentation →](Tweety-6-Structured-Argumentation.ipynb)

---

## Objectifs pédagogiques

1. Comprendre les cadres d'argumentation abstraits de Dung (AF)
2. Maîtriser les sémantiques d'acceptabilité (grounded, preferred, stable, complete)
3. Explorer la sémantique CF2 pour les graphes avec cycles
4. Générer des frameworks d'argumentation aléatoires

## Prérequis

Exécutez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement JVM.

> **Limitations connues (Tweety 1.28):**
> - La section 4.1.4 (Apprentissage depuis Labellisations) peut lever une `ClassCastException` - Bug interne Tweety
> - Les sections 4.1.1 à 4.1.3 fonctionnent correctement

In [6]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety + Outils ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil
import platform

# === Configuration COMPLETE des outils externes ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
    "EPROVER": "",
    "SAT_SOLVER_PYTHON": "",
    "MARCO": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str:
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# --- Auto-detection des outils ---
system = platform.system()
exe_suffix = ".exe" if system == "Windows" else ""

# 1. Clingo (ASP solver) - Tweety attend le REPERTOIRE
for cp in [shutil.which("clingo"), pathlib.Path(f"ext_tools/clingo/clingo{exe_suffix}"),
           pathlib.Path(f"../ext_tools/clingo/clingo{exe_suffix}")]:
    if cp and (isinstance(cp, str) or cp.exists()):
        parent = pathlib.Path(cp).parent if isinstance(cp, str) else cp.parent
        EXTERNAL_TOOLS["CLINGO"] = str(parent.resolve())
        break

# 2. SPASS (Modal logic prover)
for sp in [shutil.which("SPASS"), pathlib.Path(f"ext_tools/spass/SPASS{exe_suffix}"),
           pathlib.Path(f"../ext_tools/spass/SPASS{exe_suffix}")]:
    if sp and (isinstance(sp, str) or sp.exists()):
        EXTERNAL_TOOLS["SPASS"] = str(pathlib.Path(sp).resolve()) if isinstance(sp, pathlib.Path) else sp
        break

# 3. EProver (FOL theorem prover)
for ep in [shutil.which("eprover"), pathlib.Path(f"../ext_tools/EProver/eprover{exe_suffix}"),
           pathlib.Path(f"ext_tools/EProver/eprover{exe_suffix}")]:
    if ep:
        ep_path = pathlib.Path(ep) if isinstance(ep, str) else ep
        if ep_path.exists():
            EXTERNAL_TOOLS["EPROVER"] = str(ep_path.resolve())
            break

# 4. SAT Solver Python (CaDiCaL, Glucose via pySAT)
for sat in [pathlib.Path("../ext_tools/sat_solver.py"), pathlib.Path("ext_tools/sat_solver.py")]:
    if sat.exists():
        EXTERNAL_TOOLS["SAT_SOLVER_PYTHON"] = str(sat.resolve())
        break

# 5. MARCO (MUS enumerator avec Z3)
for mp in [pathlib.Path("../ext_tools/marco.py"), pathlib.Path("ext_tools/marco.py")]:
    if mp.exists():
        EXTERNAL_TOOLS["MARCO"] = str(mp.resolve())
        break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = None
    for jdk_path in [pathlib.Path("jdk-17-portable"), pathlib.Path("../Argument_Analysis/jdk-17-portable")]:
        if jdk_path.exists():
            zulu_dirs = list(jdk_path.glob("zulu*"))
            if zulu_dirs:
                jdk_portable = zulu_dirs[0]
                os.environ["JAVA_HOME"] = str(jdk_portable.resolve())
                print(f"JDK portable: {jdk_portable.name}")
                break

    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("../Argument_Analysis/libs")

        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

# === Resume des outils ===
if jvm_ready:
    print("\n--- Outils disponibles ---")
    for tool, path in EXTERNAL_TOOLS.items():
        if path:
            short_path = path.split(os.sep)[-1] if len(path) > 30 else path
            print(f"  {tool}: {short_path}")
    print(f"\nJVM prete. Outils: {sum(1 for t,p in EXTERNAL_TOOLS.items() if p)}/{len(EXTERNAL_TOOLS)}")

--- Verification JVM Tweety + Outils ---
JVM deja en cours d'execution.

--- Outils disponibles ---
  CLINGO: clingo
  SPASS: SPASS.exe
  EPROVER: eprover.exe
  SAT_SOLVER_PYTHON: sat_solver.py
  MARCO: marco.py

JVM prete. Outils: 5/5


## Partie 4 : Argumentation Abstraite et Structurée
<a id="partie4"></a>

Nous entrons maintenant au cœur de l'argumentation computationnelle, en commençant par le cadre fondateur de Dung et en progressant vers des approches qui prennent en compte la structure interne des arguments. Cette partie est souvent plus stable car elle repose sur des modules centraux de Tweety.

### 4.1 Cadres d'Argumentation Abstraits (Dung)
<a id="4.1"></a>

Le cadre de Dung (Abstract Argumentation Framework - AAF), introduit par Phan Minh Dung en 1995, est la base de nombreuses extensions en argumentation computationnelle.

**Structure d'un AAF:**
* Un ensemble d'**Arguments** (`Argument`) considérés comme des entités atomiques (boîtes noires)
* Une relation d'**Attaque** (`Attack`) binaire entre arguments
* L'ensemble forme une **Théorie de Dung** (`DungTheory`)

**Exemple visuel - Graphe d'argumentation:**
```
    AF1: a1 <-> b1 -> c1

    (a1) <-----> (b1) -----> (c1)
      ^           |            
      |           |            
      +-----------+            

    AF2: Cycle a -> b -> c -> a

    (a) -----> (b)
      ^         |
      |         v
      +------ (c)
```

**L'objectif:** Déterminer des ensembles d'arguments collectivement acceptables, appelés **extensions**.

### Hiérarchie des Sémantiques de Dung

```
                    Conflict-Free (CF)
                          |
                          v
                    Admissible (ADM)
                          |
                          v
                    Complete (COM)
                     /    |    \
                    v     v     v
             Grounded  Preferred  Stable
               (GR)     (PRF)     (ST)
```

| Sémantique | Définition | Unicité | Existence |
|------------|------------|---------|-----------|
| **Conflict-Free** | Aucun argument n'attaque un autre de l'extension | Multiple | Toujours |
| **Admissible** | CF + défend tous ses membres | Multiple | Toujours (au moins {}) |
| **Complete** | ADM + contient tous les arguments qu'elle défend | Multiple | Toujours |
| **Grounded** | Extension complète minimale (sceptique) | **Unique** | Toujours |
| **Preferred** | Extension complète maximale (crédule) | Multiple | Toujours |
| **Stable** | CF + attaque tous les autres | Multiple | Pas toujours! |
| **CF2** | Gère les cycles via décomposition SCC | Multiple | Toujours |

**Intuition:**
- **Grounded**: Position la plus prudente (sceptique), ne défend que ce qui est indéfendable
- **Preferred**: Positions crédibles maximales (peut y en avoir plusieurs)
- **Stable**: Position complète sans zone grise (peut ne pas exister avec des cycles)

**Classes Tweety:**
- `DungTheory`: Structure du graphe
- `Argument`, `Attack`: Composants de base
- `SimpleGroundedReasoner`, `SimplePreferredReasoner`, `SimpleStableReasoner`: Calculateurs d'extensions

In [7]:
# --- 4.1.1 Cadres d'Argumentation Abstraits (Dung) : Bases et Sémantiques ---
print("\n--- 4.1.1 Cadres d'Argumentation Abstraits (Dung) : Bases et Sémantiques ---")

# Vérifier si la JVM est prête
if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée. Impossible de continuer cet exemple.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple Dung...")
    try:
        # Imports nécessaires pour Dung
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        # Importer plusieurs raisonneurs
        from org.tweetyproject.arg.dung.reasoner import (
            SimpleGroundedReasoner, SimpleStableReasoner, SimplePreferredReasoner,
            SimpleCompleteReasoner, SimpleAdmissibleReasoner, SimpleConflictFreeReasoner
        )
        # Importer Semantics si besoin pour certains raisonneurs plus avancés (non utilisés ici)
        # from org.tweetyproject.arg.dung.semantics import Semantics
        from java.util import Collection # Pour vérifier taille retour

        print("✔️ Imports Dung réussis.")

        # --- Exemple 1: A <-> B -> C ---
        af1 = DungTheory()
        a1 = Argument("a1"); b1 = Argument("b1"); c1 = Argument("c1")
        # Ajouter des arguments
        af1.add(a1); af1.add(b1); af1.add(c1)
        # Ajouter des attaques
        af1.add(Attack(a1, b1)); af1.add(Attack(b1, a1)); af1.add(Attack(b1, c1))

        print("\n--- AF1: a1 <-> b1 -> c1 ---")
        print("Cadre :", af1)

        # Calculer les extensions pour différentes sémantiques
        print("\nCalcul des extensions pour AF1:")
        try:
            # Grounded: getModel() retourne une Extension, getModels() retourne une Collection<Extension>
            grounded_ext = SimpleGroundedReasoner().getModel(af1) # Plus direct pour grounded
            print(f" - Grounded   : {{{grounded_ext}}}") # Afficher comme un ensemble pour la cohérence
        except Exception as e: print(f"   Erreur Grounded: {e}")
        try:
            stable_exts = SimpleStableReasoner().getModels(af1)
            print(f" - Stable     ({stable_exts.size()}):", stable_exts)
        except Exception as e: print(f"   Erreur Stable: {e}")
        try:
            preferred_exts = SimplePreferredReasoner().getModels(af1)
            print(f" - Preferred  ({preferred_exts.size()}):", preferred_exts)
        except Exception as e: print(f"   Erreur Preferred: {e}")
        try:
            complete_exts = SimpleCompleteReasoner().getModels(af1)
            print(f" - Complete   ({complete_exts.size()}):", complete_exts)
        except Exception as e: print(f"   Erreur Complete: {e}")
        try:
            admissible_exts = SimpleAdmissibleReasoner().getModels(af1)
            # Convertir la Collection Java en liste Python pour len()
            # Note: Cela peut être coûteux si la collection est énorme
            admissible_list = list(admissible_exts)
            print(f" - Admissible ({len(admissible_list)}):", admissible_exts) # Afficher la collection Java
        except Exception as e: print(f"   Erreur Admissible: {e}")
        try:
            conflict_free_exts = SimpleConflictFreeReasoner().getModels(af1)
            cf_list = list(conflict_free_exts)
            print(f" - ConflictFree ({len(cf_list)}):", conflict_free_exts) # Attention, peut être très grand
        except Exception as e: print(f"   Erreur ConflictFree: {e}")


        # --- Exemple 2: Cycle a->b->c->a ---
        af_cycle = DungTheory()
        a_cy = Argument("a_cy"); b_cy = Argument("b_cy"); c_cy = Argument("c_cy")
        af_cycle.add(a_cy); af_cycle.add(b_cy); af_cycle.add(c_cy)
        af_cycle.add(Attack(a_cy, b_cy)); af_cycle.add(Attack(b_cy, c_cy)); af_cycle.add(Attack(c_cy, a_cy))

        print("\n--- AF Cycle: a -> b -> c -> a ---")
        print("Cadre :", af_cycle)
        print("\nCalcul des extensions pour AF Cycle:")
        try:
            print(" - Grounded   :", SimpleGroundedReasoner().getModel(af_cycle)) # {}
        except Exception as e: print(f"   Erreur Grounded: {e}")
        try:
            stable_exts_cy = SimpleStableReasoner().getModels(af_cycle)
            print(f" - Stable     ({stable_exts_cy.size()}):", stable_exts_cy) # {}
        except Exception as e: print(f"   Erreur Stable: {e}")
        try:
            preferred_exts_cy = SimplePreferredReasoner().getModels(af_cycle)
            print(f" - Preferred  ({preferred_exts_cy.size()}):", preferred_exts_cy) # [{}]
        except Exception as e: print(f"   Erreur Preferred: {e}")
        try:
            complete_exts_cy = SimpleCompleteReasoner().getModels(af_cycle)
            print(f" - Complete   ({complete_exts_cy.size()}):", complete_exts_cy) # [{}]
        except Exception as e: print(f"   Erreur Complete: {e}")


    except ImportError as e:
        print(f"❌ Erreur d'import pour l'Argumentation Abstraite : {e}")
        print("   Vérifiez le JAR 'org.tweetyproject.arg.dung'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple Dung: {e_java.message()}")
        # print(e_java.stacktrace()) # Décommenter pour trace Java complète
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple Dung: {e_gen}")
        import traceback
        traceback.print_exc()


--- 4.1.1 Cadres d'Argumentation Abstraits (Dung) : Bases et Sémantiques ---
ℹ️ JVM prête. Exécution de l'exemple Dung...
✔️ Imports Dung réussis.

--- AF1: a1 <-> b1 -> c1 ---
Cadre : <{ a1, c1, b1 },[(b1,c1), (b1,a1), (a1,b1)]>

Calcul des extensions pour AF1:
 - Grounded   : {{}}
 - Stable     (2): [{a1,c1}, {b1}]
 - Preferred  (2): [{a1,c1}, {b1}]
 - Complete   (3): [{a1,c1}, {b1}, {}]
 - Admissible (4): [{a1,c1}, {b1}, {}, {a1}]
 - ConflictFree (5): [{c1}, {a1,c1}, {b1}, {}, {a1}]

--- AF Cycle: a -> b -> c -> a ---
Cadre : <{ a_cy, c_cy, b_cy },[(c_cy,a_cy), (a_cy,b_cy), (b_cy,c_cy)]>

Calcul des extensions pour AF Cycle:
 - Grounded   : {}
 - Stable     (0): []
 - Preferred  (1): [{}]
 - Complete   (1): [{}]


#### 4.1.2 Sémantique CF2
<a id="4.1.2"></a>

La sémantique CF2 ([Baroni, Giacomin, Guida, *SCC-recursive semantics for abstract argumentation frameworks*, 2005](https://link.springer.com/chapter/10.1007/11557791_3)) est une alternative aux sémantiques classiques qui gère différemment les cycles, notamment les cycles impairs. Elle se base sur une décomposition du graphe d'argumentation en Composantes Fortement Connexes (SCCs). Tweety fournit le `SccCF2Reasoner`.

In [8]:
# --- 4.1.2 Sémantique CF2 ---
print("\n--- 4.1.2 Sémantique CF2 ---")

# Vérifier si la JVM est prête
if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée. Impossible de continuer cet exemple.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple CF2...")
    try:
        # Imports (peuvent être déjà faits, mais sécurité)
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        from org.tweetyproject.arg.dung.reasoner import SccCF2Reasoner
        from org.tweetyproject.arg.dung.semantics import Extension # Pour type hinting éventuel
        from java.util import Collection

        print("✔️ Imports Dung (pour CF2) réussis.")

        # --- Exemple AF2 : Cycle a->b->c->d->e->a, e->f ---
        # (Recréation pour autonomie de la cellule)
        af2 = DungTheory()
        args_cf2_map = {name: Argument(name) for name in "abcdef"} # Utiliser un dictionnaire
        for arg in args_cf2_map.values(): af2.add(arg)

        attacks_cf2 = [("a","b"), ("b","c"), ("c","d"), ("d","e"), ("e","a"), ("e","f")]
        for s, t in attacks_cf2:
             # S'assurer que les arguments existent avant d'ajouter l'attaque
             if s in args_cf2_map and t in args_cf2_map:
                  af2.add(Attack(args_cf2_map[s], args_cf2_map[t]))
             else:
                  print(f"WARN: Argument(s) non trouvé(s) pour l'attaque ({s},{t})")

        print("\n--- AF pour CF2 : Cycle a->b->c->d->e->a, e->f ---")
        print("Cadre :", af2)

        # --- Raisonnement CF2 ---
        cf2_reasoner = SccCF2Reasoner()
        print("\nCalcul des extensions CF2:")
        try:
            cf2_extensions_collection = cf2_reasoner.getModels(af2) # Retourne Collection<Extension>
            if cf2_extensions_collection.isEmpty():
                 print("  (Aucune extension CF2 trouvée)")
            else:
                 # Itérer sur la collection Java
                 ext_iterator = cf2_extensions_collection.iterator()
                 count = 0
                 while ext_iterator.hasNext():
                      ext = ext_iterator.next()
                      # Pour un affichage plus propre des arguments dans l'extension:
                      args_in_ext = ", ".join(sorted([str(arg.getName()) for arg in ext]))
                      print(f"  - {{{args_in_ext}}}")
                      count += 1
                 print(f"  ({count} extension(s) trouvée(s))")

        except jpype.JException as e_cf2_java:
             print(f"  ❌ Erreur Java lors du raisonnement CF2: {e_cf2_java.message()}")
             # print(e_cf2_java.stacktrace())
        except Exception as e_cf2_py:
              print(f"  ❌ Erreur Python lors du raisonnement CF2: {e_cf2_py}")

    # Gestion globale
    except ImportError as e:
        print(f"❌ Erreur d'import pour CF2 : {e}. Vérifiez le JAR 'arg.dung'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple CF2: {e_java.message()}")
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple CF2: {e_gen}")
        import traceback
        traceback.print_exc()


--- 4.1.2 Sémantique CF2 ---
ℹ️ JVM prête. Exécution de l'exemple CF2...
✔️ Imports Dung (pour CF2) réussis.

--- AF pour CF2 : Cycle a->b->c->d->e->a, e->f ---
Cadre : <{ a, b, c, d, e, f },[(b,c), (d,e), (e,a), (a,b), (c,d), (e,f)]>

Calcul des extensions CF2:
  - {b, e}
  - {c, e}
  - {a, c, f}
  - {a, d, f}
  - {b, d, f}
  (5 extension(s) trouvée(s))


#### 4.1.3 Génération de Cadres d'Argumentation
<a id="4.1.3"></a>

Tweety fournit des outils pour générer aléatoirement des cadres d'argumentation abstraits (`DungTheory`), ce qui est utile pour les tests, benchmarks ou simulations. `DefaultDungTheoryGenerator` est un générateur simple paramétrable.

In [9]:
# --- 4.1.3 Génération de Cadres Dung ---
print("\n--- 4.1.3 Génération de Cadres Dung ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple de génération...")
    try:
        # Imports
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        from org.tweetyproject.arg.dung.util import DefaultDungTheoryGenerator, DungTheoryGenerationParameters

        print("✔️ Imports pour génération Dung réussis.")

        # Paramètres de génération
        params = DungTheoryGenerationParameters()
        params.numberOfArguments = 6
        params.attackProbability = 0.25
        # CORRECTION: La ligne suivante est retirée car le champ n'existe probablement plus
        # params.allowSelfAttacks = False
        print(f"ℹ️ Paramètres de génération: {params.numberOfArguments} args, proba_attaque={params.attackProbability} (comportement par défaut pour auto-attaques)")

        generator = DefaultDungTheoryGenerator(params)

        # Générer UN cadre
        generated_af = generator.next()

        print("\nCadre généré aléatoirement :")
        print(f"  Arguments: {generated_af.getNodes()}")
        print(f"  Attaques : {generated_af.getAttacks()}")
        print(f"  Représentation compacte: {generated_af}")

    except ImportError as e:
        print(f"❌ Erreur d'import pour la génération Dung : {e}. Vérifiez le JAR 'arg.dung'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple génération Dung: {e_java.message()}")
        # print(e_java.stacktrace())
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple génération Dung: {e_gen}")
        import traceback
        traceback.print_exc()


--- 4.1.3 Génération de Cadres Dung ---
ℹ️ JVM prête. Exécution de l'exemple de génération...
✔️ Imports pour génération Dung réussis.
ℹ️ Paramètres de génération: 6 args, proba_attaque=0.25 (comportement par défaut pour auto-attaques)

Cadre généré aléatoirement :
  Arguments: <{ a1, a2, a3, a4, a5, a0 },[(a3,a4), (a3,a5), (a2,a4), (a2,a5), (a0,a3), (a0,a4), (a5,a3), (a1,a0)]>
  Attaques : [(a3,a4), (a3,a5), (a2,a4), (a2,a5), (a0,a3), (a0,a4), (a5,a3), (a1,a0)]
  Représentation compacte: <{ a1, a2, a3, a4, a5, a0 },[(a3,a4), (a3,a5), (a2,a4), (a2,a5), (a0,a3), (a0,a4), (a5,a3), (a1,a0)]>


#### 4.1.4 Apprentissage de Cadres d'Argumentation (depuis Labellisations)
<a id="4.1.4"></a>

L'apprentissage de cadre vise à reconstruire un cadre d'argumentation (ou un ensemble de cadres possibles) à partir d'informations partielles, typiquement des exemples de labellisations d'arguments (IN, OUT, UNDEC) selon une certaine sémantique. Tweety propose `SimpleAFLearner` pour cela.

In [10]:
# --- 4.1.4 Apprentissage de Cadres Dung (depuis Labellisations) ---
print("\n--- 4.1.4 Apprentissage de Cadres Dung ---")

print("!! SECTION COMMENTÉE !!")
print("L'exécution de cet exemple échoue en raison d'une ClassCastException interne à Tweety")
print("(Tautology cannot be cast to AssociativePlFormula) lors de l'appel à getLabeling/learnLabeling")
print("pour le cadre d'argumentation spécifique utilisé ici. Ceci semble être un bug interne.")

# if not jvm_ready:
#     print("❌ ERREUR: JVM non démarrée.")
# else:
#     print("ℹ️ JVM prête. Exécution de l'exemple d'apprentissage...")
#     try:
#         # Imports
#         import jpype
#         from jpype.types import *
#         from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
#         from org.tweetyproject.arg.dung.learning import SimpleAFLearner
#         from org.tweetyproject.arg.dung.learning.syntax import Entity
#         from org.tweetyproject.arg.dung.semantics import Semantics
#         # Import de Label retiré (correction précédente)
#
#         print("✔️ Imports pour apprentissage Dung réussis.")
#
#         # 1. Définir le cadre "caché"
#         hidden_af = DungTheory()
#         a_learn = Argument("a"); b_learn = Argument("b"); c_learn = Argument("c")
#         hidden_af.add(a_learn); hidden_af.add(b_learn); hidden_af.add(c_learn)
#         hidden_af.add(Attack(a_learn, b_learn)); hidden_af.add(Attack(b_learn, a_learn)); hidden_af.add(Attack(b_learn, c_learn))
#         print("\nCadre caché (à apprendre):", hidden_af)
#
#         # 2. Créer l'Oracle
#         oracle = Entity(hidden_af)
#         arguments_known = oracle.getArguments()
#         print("Arguments connus par l'apprenant:", arguments_known)
#
#         # 3. Créer l'Apprenant
#         learner = SimpleAFLearner(arguments_known)
#         print(f"État initial apprenant: {learner.getNumberOfFrameworks()} cadres compatibles possibles.")
#
#         # 4. Apprendre depuis des labellisations
#         # Labellisation Stable
#         print("\nApprentissage depuis labellisation Stable:")
#         try:
#              labeling_stable = oracle.getLabeling(Semantics.STABLE_SEMANTICS) # C'est ici que l'erreur se produit
#              print(f"  Oracle fournit Labellisation STABLE: {labeling_stable}")
#              learner.learnLabeling(labeling_stable)
#              print(f"  Après STABLE: {learner.getNumberOfFrameworks()} cadres compatibles restants.")
#         except jpype.JException as e_stable:
#               print(f"  Erreur lors de l'obtention/apprentissage de la labellisation Stable: {e_stable.message()}")
#               # Afficher la stacktrace Java peut aider à confirmer l'erreur interne
#               # print(e_stable.stacktrace())
#
#         # Labellisation Conflict-Free (pourrait aussi échouer)
#         print("\nApprentissage depuis labellisation Conflict-Free:")
#         try:
#              labeling_cf = oracle.getLabeling(Semantics.CONFLICTFREE_SEMANTICS)
#              print(f"  Oracle fournit Labellisation CONFLICT_FREE: {labeling_cf}")
#              learner.learnLabeling(labeling_cf)
#              print(f"  Après CONFLICT_FREE: {learner.getNumberOfFrameworks()} cadres compatibles restants.")
#         except jpype.JException as e_cf:
#              print(f"  Erreur lors de l'obtention/apprentissage de la labellisation ConflictFree: {e_cf.message()}")
#
#
#         # 5. Obtenir le(s) cadre(s) appris
#         # Ce code ne sera probablement pas atteint
#         learned_af = learner.getModel()
#         print("\nCadre appris (un des cadres compatibles) :")
#         print(str(learned_af))
#
#     # ... (Blocs except ImportError, JException, Exception comme avant) ...
#     except ImportError as e:
#        print(f"❌ Erreur d'import pour l'apprentissage Dung : {e}.")
#     except jpype.JException as e_java:
#        print(f"❌ Erreur Java générale dans l'exemple apprentissage Dung: {e_java.message()}")
#     except Exception as e_gen:
#        print(f"❌ Erreur Python inattendue dans l'exemple apprentissage Dung: {e_gen}")
#        import traceback
#        traceback.print_exc()


--- 4.1.4 Apprentissage de Cadres Dung ---
!! SECTION COMMENTÉE !!
L'exécution de cet exemple échoue en raison d'une ClassCastException interne à Tweety
(Tautology cannot be cast to AssociativePlFormula) lors de l'appel à getLabeling/learnLabeling
pour le cadre d'argumentation spécifique utilisé ici. Ceci semble être un bug interne.


---

## Résumé

Ce notebook a couvert:
- **Cadres de Dung (AF)**: Arguments, attaques, représentation graphique
- **Sémantiques d'acceptabilité**: Grounded, preferred, stable, complete
- **Sémantique CF2**: Traitement des cycles via décomposition SCC
- **Génération**: Création aléatoire de frameworks

## Prochaines étapes

Le notebook suivant explore l'argumentation structurée avec ASPIC+, DeLP, ABA et ASP.

---

**Navigation**: [← Tweety-4-Belief-Revision](Tweety-4-Belief-Revision.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-6-Structured-Argumentation →](Tweety-6-Structured-Argumentation.ipynb)